In [2]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import tqdm

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from itertools import chain

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import detection_utils as utils
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

1.9.0+cu102 True


In [3]:
# Register the datasets
def get_data(dataroot, file_name):
    def ret_data():
        output_path = os.path.join(dataroot, file_name)
        print(output_path)
        assert os.path.exists(output_path)
        with open(output_path, 'r') as fd:
            data = json.load(fd)
        fd.close()
        return data
    return ret_data

In [4]:
dataroot = '/usr0/tma1/datasets/nuimages/detectron_data'
train_file = 'v1.0-train_trafficcone_detectron.json'
val_file = 'val_val_trafficcone_detectron.json'
test_file = 'val_test_trafficcone_detectron.json'

In [5]:
category_dict = {0: 'traffic_cones'}
data_train = DatasetCatalog.register("nuimages_train_trafficone", get_data(dataroot, train_file))
MetadataCatalog.get("nuimages_train_trafficone").thing_classes = category_dict
data_val = DatasetCatalog.register("nuimages_val_trafficone", get_data(dataroot, val_file))
MetadataCatalog.get("nuimages_val_trafficone").thing_classes = category_dict
data_test = DatasetCatalog.register("nuimages_test_trafficone", get_data(dataroot, test_file))
MetadataCatalog.get("nuimages_test_trafficone").thing_classes = category_dict

In [6]:
date_time = '2021-10-26-01-38-57'
gt_folder = '/usr0/tma1/datasets/nuimages/detectron_data'
gt_file = 'val_val_trafficcone_detectron.json'
gt_path = os.path.join(gt_folder, gt_file)

In [7]:
# Parameters
show_image=False
output_dir='/usr0/tma1/datasets/nuimages/object_detection_annotations'
predicted_dir='/usr0/tma1/traffic_cone_detection/output'
visualize_dataset_name = 'nuimages_test_trafficone'
scale=5.0
predicted_output_folder = os.path.join(os.path.join(predicted_dir, date_time), 'predicted_output')
if not os.path.exists(predicted_output_folder):
    os.mkdir(predicted_output_folder)
    
prediction_name = date_time + '_' + visualize_dataset_name
predicted_output_folder = os.path.join(predicted_output_folder, prediction_name)
if not os.path.exists(predicted_output_folder):
    os.mkdir(predicted_output_folder)                                    

Visualize Predicted Outputs

In [8]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.OUTPUT_DIR = os.path.join('/usr0/tma1/traffic_cone_detection/output', date_time)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0076499.pth")
# cfg.MODEL.DEVICE='cpu'
predictor = DefaultPredictor(cfg)

In [23]:
dicts = list(chain.from_iterable([DatasetCatalog.get(k) for k in [visualize_dataset_name]]))
metadata = MetadataCatalog.get(visualize_dataset_name)
for dic in tqdm.tqdm(dicts):
    im = utils.read_image(dic["file_name"], "RGB")
    outputs = predictor(im)
#     print(outputs)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    figure(figsize=(6,6), dpi=150)
    plt.imshow(out.get_image()[:, :, ::-1])
#     plt.show()
    parsed_file = dic["file_name"].split('/')[-1]
    plt.savefig(os.path.join(predicted_output_folder, parsed_file), dpi=200)

/usr0/tma1/datasets/nuimages/detectron_data/val_test_trafficcone_detectron.json


  0%|          | 0/2263 [00:00<?, ?it/s]


AttributeError: Cannot find field 'num_instances' in the given Instances!

Visualize Ground Truths

In [11]:
def output(vis, fname):
    if show_image:
        print(fname)
        img = vis.get_image()[:, :, ::-1]
        # cv2.imshow("window", vis.get_image()[:, :, ::-1])
        figure(figsize=(10,10), dpi=80)
        plt.imshow(img)
        plt.show()
    else:
        filepath = os.path.join(output_dir, fname)
        print("Saving to {} ...".format(filepath))
        vis.save(filepath)

In [12]:
dicts = list(chain.from_iterable([DatasetCatalog.get(k) for k in [visualize_dataset_name]]))
metadata = MetadataCatalog.get(visualize_dataset_name)
for dic in tqdm.tqdm(dicts):
    img = utils.read_image(dic["file_name"], "RGB")
    visualizer = Visualizer(img, metadata=metadata, scale=scale)
    vis = visualizer.draw_dataset_dict(dic)
    output(vis, os.path.basename(dic["file_name"]))

/usr0/tma1/datasets/nuimages/detectron_data/v1.0-val_trafficcone_detectron.json


  0%|          | 0/4526 [00:00<?, ?it/s]

Saving to /usr0/tma1/datasets/nuimages/object_detection_annotations/n013-2018-08-21-14-27-06+0800__CAM_FRONT__1534833683412407.jpg ...


  0%|          | 1/4526 [00:02<2:40:04,  2.12s/it]

Saving to /usr0/tma1/datasets/nuimages/object_detection_annotations/n009-2018-07-12-16-12-53-0400__CAM_BACK__1531426991637579.jpg ...


...

100%|█████████▉| 4524/4526 [2:36:12<00:04,  2.04s/it]

Saving to /usr0/tma1/datasets/nuimages/object_detection_annotations/n013-2018-08-21-14-27-06+0800__CAM_FRONT_RIGHT__1534833562020176.jpg ...


100%|█████████▉| 4525/4526 [2:36:15<00:02,  2.09s/it]

Saving to /usr0/tma1/datasets/nuimages/object_detection_annotations/n008-2018-06-05-15-07-28-0400__CAM_BACK_LEFT__1528226062897295.jpg ...


100%|██████████| 4526/4526 [2:36:17<00:00,  2.07s/it]


Evaluate Results

In [ ]:
# Precision-Recall curve
def calc_precision_recall(y_true, y_pred):
    
    # Convert predictions to series with index matching y_true
    y_pred = pd.Series(y_pred, index=y_true.index)
    
    # Instantiate counters
    TP = 0
    FP = 0
    FN = 0

    # Determine whether each prediction is TP, FP, TN, or FN
    for i in y_true.index: 
        if y_true[i]==y_pred[i]==1:
           TP += 1
        if y_pred[i]==1 and y_true[i]!=y_pred[i]:
           FP += 1
        if y_pred[i]==0 and y_test[i]!=y_pred[i]:
           FN += 1
    
    # Calculate true positive rate and false positive rate
    # Use try-except statements to avoid problem of dividing by 0
    try:
        precision = TP / (TP + FP)
    except:
        precision = 1
    
    try:
        recall = TP / (TP + FN)
    except:
        recall = 1

    return precision, recall